## 1. Init

In [3]:
import os
import os.path as op
import nltk
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
from collections import Counter

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from func import *

## 2. Feature Engineering
### 2.1 Blogs

In [ ]:
#Mine POS patterns
tagList = [
    'NN', 'CC', 'LS', 'PDT', 'POS', 'SYM', 'NNS', 'NNP', 'NNPS', 'FW', 'CD',
    'JJ', 'JJR', 'JJS', 'IN', 'TO', 'DT', 'EX', 'PRP', 'PRP$', 'WDT', 'WP',
    'WP$', 'MD', 'VB', 'VBZ', 'VBP', 'VBD', 'VBN', 'VBG', 'RB', 'RBR', 'RBS',
    'RP', 'WRB', 'UH', '.'
]

for gender in [0, 1]:
    if gender == 0:
        txtDir = './postprocess_blogs/blogs/female/'
    else:
        txtDir = './postprocess_blogs/blogs/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in range(0, len(blogs_gender)):
        m = blogs_gender[i]
        print("Processing: ", m)
        print("Files:", len(os.listdir(txtDir + m)))
        for file in os.listdir(txtDir + m):
            text = gettext(txtDir + m + '/' + file)
            sentences = nltk.sent_tokenize(text)
            CorpusPOS(sentences)

infile = open('CorpusPOS.txt', 'r')
cPOS = infile.readlines()
infile.close()
(a, b, c, d, e) = calc_probabilities(cPOS)
q1_output(a, b, c, d, e)

Prob = {}
infile = open('probabilities.txt', 'r')
prob_text = infile.readlines()

for sentence in prob_text:
    keyValPair = sentence.split(":")
    Prob[keyValPair[0]] = float(keyValPair[1][:-1])

infile.close()

posFeatures = minePOSPats(cPOS)

In [ ]:
#Feature extract
names = []
F_features = []
GRF_features = []
WC_features = []
POS_features = []
labels = []

for gender in [0, 1]:
    if gender == 0:
        txtDir = './postprocess_blogs/blogs/female/'
    else:
        txtDir = './postprocess_blogs/blogs/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in range(0, len(blogs_gender)):
        m = blogs_gender[i]
        print("Processing: ", m)
        print("Files:", len(os.listdir(txtDir + m)))
        for file in os.listdir(txtDir + m):
            name = txtDir + m + '/' + file
            text = gettext(name)
            words = getwords(text)
            sentences = nltk.sent_tokenize(text)
            words_s = split(words)
            tags = [nltk.pos_tag(word) for word in words]
            tags_s = split(tags)
            words_l = wordlemmatize(tags_s)

            F_feature = F_measure(tags_s)
            GRF_feature = Gender_Preferential_Features(words_l)
            WC_feature = Word_Classes_Feature(words_l)

            textTags = ""
            for word, tag in tags_s:
                if tag in tagList:
                    textTags = textTags + tag + " "

            POS_feature = []

            for feature in posFeatures:
                if feature in textTags:
                    POS_feature.append(1)
                else:
                    POS_feature.append(0)

            names.append(name)
            F_features.append(F_feature)
            GRF_features.append(GRF_feature)
            WC_features.append(WC_feature)
            POS_features.append(POS_feature)
            labels.append(gender)

In [ ]:
#Get single
WC_features_l = []
for i in range(len(WC_features[0])):
    n = i
    WC_features_l.append(getsingle(WC_features, n))

GRF_features_l = []
for i in range(len(GRF_features[0])):
    n = i
    GRF_features_l.append(getsingle(GRF_features, n))

POS_features_l = []
for i in range(len(POS_features[0])):
    n = i
    POS_features_l.append(getsingle(POS_features, n))

In [ ]:
#Get DataFrame
map1 = {'name': names, 'label': labels, 'F_feature': F_features}

for i in range(len(WC_features[0])):
    key = 'WC_' + str(i + 1)
    value = WC_features_l[i]
    map1[key] = value

for i in range(len(GRF_features[0])):
    key = 'GRF_' + str(i + 1)
    value = GRF_features_l[i]
    map1[key] = value

for i in range(len(POS_features[0])):
    key = 'POS_' + str(i + 1)
    value = POS_features_l[i]
    map1[key] = value

allofall = pd.DataFrame(map1)
F_features_u = np.array(F_features)
F_features_u = (F_features_u - np.mean(F_features_u)) / np.std(F_features_u)
allofall['F_feature'] = F_features_u

allofall.to_csv('allofall_blogs.csv', index=False)
genderbias = pd.read_csv('blogs_genderbias.csv')
allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
    'word ratio']
allofall.to_csv('blogs_features.csv', index=False)

### 2.2 Novels

In [ ]:
#Word count and feature extract
names = []
F_features = []
GRF_features = []
WC_features = []
labels = []
lengths = []
for gender in [0, 1]:
    if gender == 0:
        txtDir = './5000/female/'
    else:
        txtDir = './5000/male/'

    print("Processing gender: {}".format(txtDir))
    handle_gender = os.listdir(txtDir)
    print("Files:", len(handle_gender))
    for m in handle_gender:
        name = txtDir + m
        text = gettext(name)
        words = nltk.word_tokenize(text)
        tags = nltk.pos_tag(words)
        words_l = wordlemmatize(tags)

        length = len(words)
        F_feature = F_measure(tags)
        GRF_feature = Gender_Preferential_Features(words_l)
        WC_feature = Word_Classes_Feature(words_l)

        names.append(name)
        lengths.append(length)
        F_features.append(F_feature)
        GRF_features.append(GRF_feature)
        WC_features.append(WC_feature)
        labels.append(gender)

WC_features_l = []
GRF_features_l = []

for i in range(len(WC_features[0])):
    n = i
    WC_features_l.append(getsingle(WC_features, n))

for i in range(len(GRF_features[0])):
    n = i
    GRF_features_l.append(getsingle(GRF_features, n))

map1 = {
    'name': names,
    'label': labels,
    'F_feature': F_features,
    'word count': lengths
}

for i in range(len(WC_features[0])):
    key = 'WC_' + str(i + 1)
    value = WC_features_l[i]
    map1[key] = value

for i in range(len(GRF_features[0])):
    key = 'GRF_' + str(i + 1)
    value = GRF_features_l[i]
    map1[key] = value

allofall = pd.DataFrame(map1)

F_features_u = np.array(F_features)
F_features_u = (F_features_u - np.mean(F_features_u)) / np.std(F_features_u)
allofall['F_feature'] = F_features_u

allofall.to_csv('allofall_5000.csv', index=False)
genderbias = pd.read_csv('5000_genderbias.csv')
allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
    'word ratio']
allofall.to_csv('5000_features.csv', index=False)

In [ ]:
allofall['word count'].describe()

In [6]:
#shorten text
#length_list = [200, 400, 600, 800, 1200, 1600, 2000, 27850]
length_list = [10000, 20000]

for l in length_list:
    print('\nPresent length:{}'.format(l))
    for gender in [0, 1]:
        if gender == 0:
            txtDir = './5000/female/'
            saveDir = f'./5000-{l}/female/'
        else:
            txtDir = './5000/male/'
            saveDir = f'./5000-{l}/male/'
        if not op.isdir(f'./5000-{l}'):
            os.mkdir(f'./5000-{l}')
        if not op.isdir(saveDir):
            os.mkdir(saveDir)
        print("Processing gender: {}".format(txtDir))
        handle_gender = os.listdir(txtDir)
        print("Files:", len(handle_gender))
        for idx, m in enumerate(handle_gender):
            name = txtDir + m
            text = gettext(name)
            words = nltk.word_tokenize(text)
            if len(words) < (l + 1):
                continue
            words = words[:l]
            text = ''
            for word in words:
                text = text + word + ' '
            with open(op.join(saveDir, m), 'w') as f:
                f.write(text)
            if (idx + 1) % 500 == 0:
                print('Done:{}'.format(idx + 1))

Processing gender: ./5000/female/
Files: 2500
Done:500
Done:1000
Done:1500
Done:2000
Done:2500
Processing gender: ./5000/male/
Files: 2500
Done:500
Done:1000
Done:1500
Done:2500
Processing gender: ./5000/female/
Files: 2500
Done:500
Done:1000
Done:1500
Done:2000
Processing gender: ./5000/male/
Files: 2500
Done:500
Done:1000
Done:1500
Done:2500


In [4]:
#feature extract
length_list = [10000, 20000]
for l in length_list:
    names = []
    F_features = []
    GRF_features = []
    WC_features = []
    labels = []
    print('\nPresent length:{}'.format(l))
    for gender in [0, 1]:
        if gender == 0:
            txtDir = f'./5000-{l}/female/'
        else:
            txtDir = f'./5000-{l}/male/'

        print("Processing gender: {}".format(txtDir))
        handle_gender = os.listdir(txtDir)
        print("Files:", len(handle_gender))
        for m in tqdm(handle_gender):
            name = txtDir + m
            text = gettext(name)
            words = nltk.word_tokenize(text)
            tags = nltk.pos_tag(words)
            words_l = wordlemmatize(tags)

            F_feature = F_measure(tags)
            GRF_feature = Gender_Preferential_Features(words_l)
            WC_feature = Word_Classes_Feature(words_l)

            names.append(name)
            F_features.append(F_feature)
            GRF_features.append(GRF_feature)
            WC_features.append(WC_feature)
            labels.append(gender)

    WC_features_l = []
    GRF_features_l = []

    for i in range(len(WC_features[0])):
        n = i
        WC_features_l.append(getsingle(WC_features, n))

    for i in range(len(GRF_features[0])):
        n = i
        GRF_features_l.append(getsingle(GRF_features, n))

    map1 = {'name': names, 'label': labels, 'F_feature': F_features}

    for i in range(len(WC_features[0])):
        key = 'WC_' + str(i + 1)
        value = WC_features_l[i]
        map1[key] = value

    for i in range(len(GRF_features[0])):
        key = 'GRF_' + str(i + 1)
        value = GRF_features_l[i]
        map1[key] = value

    allofall = pd.DataFrame(map1)

    F_features_u = np.array(F_features)
    F_features_u = (F_features_u -
                    np.mean(F_features_u)) / np.std(F_features_u)
    allofall['F_feature'] = F_features_u

    allofall.to_csv(f'allofall_5000-{l}.csv', index=False)
    #genderbias = pd.read_csv(f'5000-{l}_genderbias.csv')
    #allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias['word ratio']
    #allofall.to_csv(f'5000-{l}_features.csv',index = False)

  0%|          | 0/2104 [00:00<?, ?it/s]


Present length:20000
Processing gender: ./5000-20000/female/
Files: 2104


  0%|          | 0/1954 [00:00<?, ?it/s]

Processing gender: ./5000-20000/male/
Files: 1954


100%|██████████| 1954/1954 [33:43<00:00,  1.11s/it]


## 3. Train

In [20]:
#train
allf = []
allf_i = []
gb = []
gb_i = []
without = []
without_i = []
featurelist = []
length_list = [10000, 20000]
for l in length_list:
    allofall = pd.read_csv(f'5000-{l}_features.csv')
    #genderbias = pd.read_csv(f'5000-{l}_genderbias.csv')
    #allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
    #    'word ratio']
    data, target = allofall.drop(
        columns=['name', 'label']).iloc[:].values, allofall['label'].values
    train_X, test_X, train_y, test_y = train_test_split(data,
                                                        target,
                                                        test_size=0.2,
                                                        random_state=42,
                                                        shuffle=True)
    data, target = pd.concat(
        (pd.DataFrame(train_X), pd.DataFrame(test_X))), pd.concat(
            (pd.DataFrame(train_y), pd.DataFrame(test_y)))
    data1 = data.iloc[:, :-2]
    data2 = data.iloc[:, -2:]
    rnd_clf = RandomForestClassifier(n_estimators=800,
                                     max_leaf_nodes=50,
                                     n_jobs=-1)

    rnd_clf.fit(data.values, target.values.reshape(-1, ))
    scores_rnd_clf_cv = cross_val_score(rnd_clf,
                                        data.values,
                                        target.values.reshape(-1, ),
                                        cv=5)
    print('\n Length:{}'.format(l))
    print('All features:')
    print(scores_rnd_clf_cv)
    print("Accuracy: %0.2f (+/- %0.2f)" %
          (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))
    allf.append(scores_rnd_clf_cv.mean())
    allf_i.append([rnd_clf.feature_importances_])

    rnd_clf.fit(data1.values, target.values.reshape(-1, ))
    scores_rnd_clf_cv = cross_val_score(rnd_clf,
                                        data1.values,
                                        target.values.reshape(-1, ),
                                        cv=5)
    print('Without Gender bias:')
    print(scores_rnd_clf_cv)
    print("Accuracy: %0.2f (+/- %0.2f)" %
          (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))
    without.append(scores_rnd_clf_cv.mean())
    without_i.append([rnd_clf.feature_importances_])

    rnd_clf.fit(data2.values, target.values.reshape(-1, ))
    scores_rnd_clf_cv = cross_val_score(rnd_clf,
                                        data2.values,
                                        target.values.reshape(-1, ),
                                        cv=5)
    print('Gender bias:')
    print(scores_rnd_clf_cv)
    print("Accuracy: %0.2f (+/- %0.2f)" %
          (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))
    gb.append(scores_rnd_clf_cv.mean())
    gb_i.append([rnd_clf.feature_importances_])


 Length:10000
All features:
[0.75       0.76777778 0.77641824 0.76195773 0.76751947]
Accuracy: 0.76 (+/- 0.02)
Without Gender bias:
[0.72111111 0.74777778 0.73859844 0.73081201 0.71746385]
Accuracy: 0.73 (+/- 0.02)
Gender bias:
[0.69222222 0.70555556 0.7163515  0.71968854 0.72080089]
Accuracy: 0.71 (+/- 0.02)

 Length:20000
All features:
[0.78448276 0.77216749 0.76108374 0.79064039 0.77037037]
Accuracy: 0.78 (+/- 0.02)
Without Gender bias:
[0.75492611 0.73891626 0.73522167 0.77955665 0.75061728]
Accuracy: 0.75 (+/- 0.03)
Gender bias:
[0.71921182 0.70197044 0.70935961 0.72536946 0.7345679 ]
Accuracy: 0.72 (+/- 0.02)


In [28]:
#length vs accuracy
df = pd.read_csv('top5_sum_all.csv')
df = df.set_index('length')

allf_ii = [0, 0]
without_ii = [0, 0]
gb_ii = [0, 0]
name = allofall.columns[:-2]
for i in range(2):
    allf_ii[i] = sorted(list(zip(name, allf_i[i][0])),
                        key=lambda x: x[1],
                        reverse=True)
    without_ii[i] = sorted(list(zip(name[:-2], without_i[i][0])),
                           key=lambda x: x[1],
                           reverse=True)
    gb_ii[i] = sorted(list(zip(name[-2:], gb_i[i][0])),
                      key=lambda x: x[1],
                      reverse=True)

allf_iii = [0, 0]
without_iii = [0, 0]
name = ['F', 'WC', 'GRF', 'bias', 'word ratio']
for i in range(2):
    temp = allf_i[i]
    handle = [0] * 5
    handle[0] = temp[0][0]
    handle[1] = sum(temp[0][1:24])
    handle[2] = sum(temp[0][24:34])
    handle[3:] = temp[0][-2:]
    handle = sorted(zip(name, handle), key=lambda x: x[1], reverse=True)
    allf_iii[i] = handle
    temp = without_i[i]
    handle = [0] * 3
    handle[0] = temp[0][0]
    handle[1] = sum(temp[0][1:24])
    handle[2] = sum(temp[0][24:34])
    handle = sorted(zip(name[:-2], handle), key=lambda x: x[1], reverse=True)
    without_iii[i] = handle

for i in range(2):
    df.iloc[7 + i, 0] = allf[i]
    df.iloc[7 + i, 3] = without[i]
    df.iloc[7 + i, 6] = gb[i]
    df.iloc[7 + i, 1] = [allf_ii[i]]
    df.iloc[7 + i, 4] = [without_ii[i]]
    df.iloc[7 + i, 7] = [gb_ii[i]]
    df.iloc[7 + i, 2] = [allf_iii[i]]
    df.iloc[7 + i, 5] = [without_iii[i]]
#df.to_csv('top5_sum.csv')

## 4. Others

In [ ]:
#10wincv
from sklearn.model_selection import cross_val_score

for i in range(1, 11):
    fname = str(10 + i) + '_win_5000_genderbias.csv'
    genderbias = pd.read_csv(fname)
    allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
        'word ratio']
    data, target = allofall.drop(columns=['name', 'label', 'word count']
                                 ).iloc[:].values, allofall['label'].values
    train_X, test_X, train_y, test_y = train_test_split(data,
                                                        target,
                                                        test_size=0.2,
                                                        random_state=42,
                                                        shuffle=True)
    data, target = pd.concat(
        (pd.DataFrame(train_X), pd.DataFrame(test_X))), pd.concat(
            (pd.DataFrame(train_y), pd.DataFrame(test_y)))

    rnd_clf = RandomForestClassifier(n_estimators=1000,
                                     max_leaf_nodes=50,
                                     n_jobs=-1)
    scores_rnd_clf_cv = cross_val_score(rnd_clf,
                                        data.values,
                                        target.values.reshape(-1, ),
                                        cv=5)
    print('Windows:%.2f'%(i))
    print(scores_rnd_clf_cv)
    print("Accuracy: %0.2f (+/- %0.2f)" %
          (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))